## Report Notebook

This is just a notebook to compare the results of the generative models on several datasets, saving the results on wandb as tables

In [9]:
import matplotlib.pyplot as plt
import wandb
import numpy as np
import os
from os.path import join as opj
from classes.VAE import VAE
from classes.VQVAE import VQVAE, VQVAE2
from classes.GAN import GAN

import tensorflow as tf
from tensorflow import keras
from tensorflow.data import AUTOTUNE
from imutils import paths


In [ ]:
wandb.login()

BS=128

#set the second GPU
os.environ["CUDA_VISIBLE_DEVICES"]="0"
print(os.environ.get("CUDA_VISIBLE_DEVICES"))

### Models path

In [16]:
vae_path="models/vae"
vqvae_path="models/vqvae_celeba"
vqvae2_path="models/vqvae2_celeba"

In [20]:
os.listdir(vqvae2_path)

FileNotFoundError: [WinError 3] Impossibile trovare il percorso specificato: 'models/vqvae2_celeba'

## CelebA comparison

In [10]:
images_dir=r"C:\Users\matte\Dataset\tor_vergata\Dataset\Img\img_align_celeba" #local
#images_dir=r"/home/matteo/NeuroGEN/Dataset/Img/img_align_celeba"



In [11]:
def load_images(imagePath):
    # read the image from disk, decode it, resize it, and scale the
    # pixels intensities to the range [0, 1]
    image = tf.io.read_file(imagePath)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, (128, 128)) / 255.0

    #eventually load other information like attributes here
    
    # return the image and the extra info
    
    
    return image

In [12]:
print("[INFO] loading image paths...")
imagePaths = list(paths.list_images(images_dir))


train_len=int(0.8*len(imagePaths))
val_len=int(0.1*len(imagePaths))
test_len=int(0.1*len(imagePaths))

train_imgs=imagePaths[:train_len]                                #      80% for training
val_imgs=imagePaths[train_len:train_len+val_len]                 #      10% for validation
test_imgs=imagePaths[train_len+val_len:]                         #      10% for testing

print(f"[TRAINING]\t {len(train_imgs)}\n[VALIDATION]\t {len(val_imgs)}\n[TEST]\t\t {len(test_imgs)}")

[INFO] loading image paths...
[TRAINING]	 113455
[VALIDATION]	 14181
[TEST]		 14183


In [15]:
#TRAINING 

train_dataset = tf.data.Dataset.from_tensor_slices(train_imgs)
train_dataset = (train_dataset
    .map(load_images, num_parallel_calls=AUTOTUNE)
    .cache()
    .repeat()
    .batch(BS)
    .shuffle(1024)
    .prefetch(AUTOTUNE)
)

ts=len(train_imgs)//BS

##VALIDATION

val_dataset = tf.data.Dataset.from_tensor_slices(val_imgs)
val_dataset = (val_dataset
    .map(load_images, num_parallel_calls=AUTOTUNE)
    .cache()
    .repeat()
    .shuffle(1024)
    .batch(BS)
    .prefetch(AUTOTUNE)
)

vs=len(val_imgs)//BS

## TEST

test_dataset = tf.data.Dataset.from_tensor_slices(test_imgs)
test_dataset = (test_dataset
    .map(load_images, num_parallel_calls=AUTOTUNE)
    .cache()
    .shuffle(1024)
    .batch(BS)
    .prefetch(AUTOTUNE)
)